In [ ]:
dat = read.csv(file="./week_4_callers.csv", header=TRUE)

In [ ]:
head(dat, n = 2)

In [ ]:
pairs(dat)

In [ ]:
plot(calls/days_active ~ isgroup2, data = dat)

In [ ]:
plot(jitter(calls) ~ jitter(isgroup2), data = dat)

In [ ]:
plot(jitter(age) ~ isgroup2, data = dat)

In [ ]:
plot(calls/days_active ~ age, data = dat)

In [ ]:
source("../shared.R")

In [ ]:
str_mod = " model {
    for (i in 1:length(calls)) {
        calls[i] ~ dpois(days_active[i] * lam[i])
        log(lam[i]) = b0 + b[1]*age[i] + b[2]*isgroup2[i]
    }
    
    b0 ~ dnorm(0, 1/10^2)
    for (j in 1:2){
        b[j] ~ dnorm(0, 1/10^2)
    }
}"

jags_dat = as.list(dat)
mod = jags_model_compile(str_mod, jags_dat, burnout = 1e3)

In [ ]:
modsim = jags_model_sample(mod, var_names_ = c('b0', 'b'), n_iter_ = 1e4)

In [ ]:
plot(modsim$samples)

In [ ]:
moddiag = jags_model_diag(samples_ = modsim$samples)

In [ ]:
X = as.matrix(modsim$samples_cmb[,c('b0', 'b[1]', 'b[2]')])

In [ ]:
head(X)

In [ ]:
u = c(1, 29, 1)

In [ ]:
lambda_hat = c(exp(X %*% t(t(u))))

In [ ]:
calls_hat = rpois(lambda = 30*lambda_hat, n = 1e4)

In [ ]:
mean(calls_hat >= 3)

In [ ]:
hist(lambda_hat)

## Quiz

`a new 29 year old customer from Group 2 whose account is active for 30 days.`

In [ ]:
u = c(1, 20, 1)


In [ ]:
rep(u, times = 2, length.out = 3)

## Model checking

In [ ]:
summary(modsim$samples_cmb)

In [ ]:
params = apply(modsim$samples_cmb, 2, mean)
params[c('b[1]', 'b[2]')]

In [ ]:
X = as.matrix(dat[, c('age', 'isgroup2')])

In [ ]:
lam_hat = exp( params["b0"] + X %*% params[c('b[1]', 'b[2]')])
yhat = lam_hat*dat[,'days_active']

In [ ]:
resid  = dat[,'calls'] - yhat

#
 `number of calls by a new 29 year old customer from Group 2 whose account is active for 30 days. `

In [ ]:
z = c(1, 30, 1) # age 30 is group2 true

In [ ]:
X = modsim$samples_cmb[,c('b0', 'b[1]', 'b[2]')]

In [ ]:
head(X)

In [ ]:
as.matrix(X) %*% z

In [ ]:
head(X %*% t(z))